<!--
Doc Writer email@nixdabei.de
v0.0.1, 2022-02-19
-->
[Home](../index.ipynb) /  Leistungstreiber L9110: Motor ansteuern & Audio
***
# Leistungstreiber L9110: Motor ansteuern & Audio
## Infos

|||
|---|---|
| Kanäle | 2 |
| Betriebsspannung | 2,5...12 V |
| Strom pro Kanal | 800 mA |
| IC| HG7881 (L9110) |


* [DayPower: Motorantriebsmodul Motor-9110](doc/D810572B.PDF) (local copy). Source: [pollin.de](https://www.pollin.de/p/motorantriebsmodul-daypower-motor-9110-810572https://www.pollin.de/p/motorantriebsmodul-daypower-motor-9110-810572)
* [IC: specs](doc/L9110_EN.pdf) (local copy)
* [Modul: Schlatplan](doc/LC-Motor-9110.pdf) (local copy)

***

<!--
| Farbe | Aufgabe |
| --- | --- |
| X | Masse/Ground |
| Y | Power |
| Z| Signal |-->


## Audio (Tonausgabe auf Lautsprecher)
### Space Sound
#### Prezedural

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

import time
import machine

machine.Pin( 12, machine.Pin.OUT ).off()
pwmTone = machine.PWM( machine.Pin(13), freq=55, duty=512 )

        
for i in range( 7 ) :
    for f in range( 55, 1000 ):
        pwmTone.freq( f )
        time.sleep( 0.001 )

pwmTone.duty( 0 )


#### Klasse: SpaceSound

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

import time
import machine

class SpaceSound :
    def __init__( self, _iPinA, _iPinB ) :
        self._iPinA      = _iPinA
        self._iPinB      = _iPinB
        self._isOn       = False
        self._iDutyOn    = 512
        
        machine.Pin( self._iPinA, machine.Pin.OUT ).off()
        self._pwmTone = machine.PWM( machine.Pin(self._iPinB), freq=440, duty=0 )

    def on( self ) :
        self._pwmTone.duty( self._iDutyOn )
        self._isOn = True

    def off( self ) :
        self._pwmTone.duty( 0 )
        self._isOn = False

    def setFrequency( self, _iFreq ) :
        if self._isOn == False :
            self.on()

        self._pwmTone.freq( _iFreq )

        
    def play( self ) :
        self.on()
        
        for i in range( 7 ) :
            for f in range( 55, 1000 ):
                self._pwmTone.freq( f ) # as fast as procedural!
                #self.setFrequency( f ) # function call is slow!
                time.sleep( 0.001 )
        
        self.off()

        
SpaceSound( 12, 13 ).play()

### Klasse: Audio
Spielt einen Ton

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

import machine
import time

class Audio :
    def __init__( self, _iPinA, _iPinB ) :
        self._iPinA   = _iPinA
        self._iPinB   = _iPinB
        self._isOn    = False
        self._iDutyOn = 512
        
        self._lLastMillis     = 0
        self._aDurationMillis = [0,0]
        self._iUDurationIndex = -1
        
        machine.Pin( self._iPinA, machine.Pin.OUT ).off()
        self._pwmTone = machine.PWM( machine.Pin(self._iPinB), freq=440, duty=0 )

    def on( self ) :
        self._pwmTone.duty( self._iDutyOn )
        self._isOn = True

    def off( self ) :
        self._pwmTone.duty( 0 )
        self._isOn = False

        
    def play( self, _fFrequency, _lDurationMillis, _lPauseMillis ) :
        if self._iUDurationIndex < 0 :
            self._lLastMillis = time.ticks_ms()
            self._iUDurationIndex = 0
            self._aDurationMillis[0] = _lDurationMillis
            self._aDurationMillis[1] = _lPauseMillis

            if ( self._isOn == False ) :
                self.on()

            self._pwmTone.freq( _fFrequency )
        else :
            if time.ticks_diff(time.ticks_ms(), self._lLastMillis) > self._aDurationMillis[self._iUDurationIndex] :
                self.off()
                self._iUDurationIndex += 1
                if self._iUDurationIndex > 1 :
                    self._iUDurationIndex = -1
                    


### Sample

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

audio = Audio( 12, 13 )

try:
    while True:
        audio.play( 1000, 100, 900 )


except KeyboardInterrupt:
    print( "Interrupted by user" )

audio.off()
print( "Done." )

### Klasse: SimpleMusic

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

import machine
import math
import time

class SimpleMusic :
    def __init__( self, _iPinA, _iPinB ) :
        self._iPinA      = _iPinA
        self._iPinB      = _iPinB

        machine.Pin( self._iPinA, machine.Pin.OUT ).off()
        self._pwmTone = machine.PWM( machine.Pin(self._iPinB), freq=440, duty=0 )
        self._mapTone = {'c': 0,'d': 2,'e': 4, 'f': 5, 'g': 7, 'a': 9, 'h': 11 }
        self._fSpeed      = 0.25
        self._fTranspose  = 0
        self._fHarmonic   = math.log(2)/12
        
    def setOff( self ) :
        self._pwmTone.duty( 0 )
        
    
    def setFrequency( self, _iFreq ) :
        if self._pwmTone.duty() == 0 :
            self._pwmTone.duty( 512 )

        self._pwmTone.freq( _iFreq )
    
    def setSpeed( self, _fSpeed ) :
        self._fSpeed  = _fSpeed

    def setTranspose( self, _fTranspose ) :
        self._fTranspose  = _fTranspose
        
    def playTone( self, _strTone ) :
        iTone   = self._mapTone[_strTone[0]]
        iOctave = _strTone[1]
        if   iOctave == '#' :
            iTone += 1
            iOctave = int(_strTone[2])
        elif iOctave == '$' :
            iTone -= 1
            iOctave = int(_strTone[2])
        else :
            iOctave = int(iOctave)
            
        #iFrequency = int( 22.5*math.exp( (self._fShift + iOctave*12 + iTone)/12*math.log( 2 ) ) )
        #print( "iTone: {} iOctave: {} self._fSpeed: {} self._fShift: {} iFrequency: {}".format(iTone,iOctave,self._fSpeed,self._fShift,iFrequency) )
        # A4 = A'' = 440Hz
        #iFrequency = int( 22.5*math.exp( (self._fShift + iOctave*12 + iTone)/12*math.log( 2 ) ) )
        self.setFrequency( int( 22.5*math.exp( (self._fTranspose + iOctave*12 + iTone)*self._fHarmonic ) ) ) # /12*math.log( 2 ) ) ) )

    def playSong( self, _strSong ) :
        aNotes = _strSong.split(",")
        for strNote in aNotes :
            aNote = strNote.split()
            if aNote[0] == "s" :
                self.setSpeed( float(aNote[1]) )
            elif aNote[0] == "t" :
                self.setTranspose( float(aNote[1]) )
            elif aNote[0] == "p" :
                time.sleep( float(aNote[1])*self._fSpeed )
            else :
                self.playTone( aNote[0] )
                fTime = float(aNote[1])*self._fSpeed
                time.sleep( fTime*0.9 )
                self.setOff()
                time.sleep( fTime*0.1 )
            
            

### Sample

In [ ]:
import time

music = SimpleMusic( 12, 13 )


#for i in range( 7 ) :
#    for f in range( 55, 1000 ):
#        music.setTone( f )
#        time.sleep( 0.001 )

#for i in range( 48 ) :
#    music.setTone( i )
#    time.sleep( 0.25 )

#song = """t 0.25,
#        c2 1, d2 1, e2 1, f2 1,
#        g2 2, g2 2,
#        a2 1, a2 1, a2 1, a2 1,
#        g2 4,
#        a2 1, a2 1, a2 1, a2 1,
#        g2 4,
#        f2 1, f2 1, f2 1, f2 1,
#        e2 2, e2 2,
#        d2 1, d2 1, d2 1, d2 1,
#        c2 4,
#        c3 1, c#3 1, d3 1, d#3 1"""


song = """s 0.11, t 0,
        c2 1, d2 1, e2 1, f2 1,
        g2 2, g2 2,
        a2 1, a2 1, a2 1, a2 1,
        g2 4,
        a2 1, a2 1, a2 1, a2 1,
        g2 4,
        f2 1, f2 1, f2 1, f2 1,
        e2 2, e2 2,
        d2 1, d2 1, d2 1, d2 1,
        c2 4,
        p 4,
        c3 0.25, c#3 0.25, d3 0.25, d#3 0.25, e3 0.25, f3 0.25, f#3 0.25, g3 0.25, g#3 0.25, a3 0.25, a#3 0.25, h3 0.25, c4 0.25"""

music.playSong( song )



## Motor ansteuern
### Klasse: Motor

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

In [ ]:
import machine

class Motor :
    def __init__( self, _iPinA, _iPinB , _iFrequency = 1000, _iDutyMinLeft = 0, _iDutyMaxLeft = 1023, _iDutyMinRight = 0, _iDutyMaxRight = 1023, _fSpeedMaxLeft = 500, _fSpeedMaxRight = 500 ) :
        self._iFrequency    = _iFrequency
        self._fScaleLeft    = -(_iDutyMaxLeft -_iDutyMinLeft )/_fSpeedMaxLeft
        self._fScaleRight   =  (_iDutyMaxRight-_iDutyMinRight)/_fSpeedMaxRight
        self._iDutyMinLeft  = _iDutyMinLeft
        self._iDutyMinRight = _iDutyMinRight
        
        self._fCurSpeed     = 0
        
        self._pwmA = machine.PWM( machine.Pin(_iPinA), freq=self._iFrequency, duty=0)
        self._pwmB = machine.PWM( machine.Pin(_iPinB), freq=self._iFrequency, duty=0)
        
        self._pwmDir   = self._pwmA
        self._pwmSpeed = self._pwmB
        
    def speedToDuty( self, _fSpeed ) :
        if _fSpeed == 0 :
            return 0
        elif _fSpeed < 0 :
            return int(self._iDutyMinLeft  + _fSpeed*self._fScaleLeft +0.5)
        else :
            return int(self._iDutyMinRight + _fSpeed*self._fScaleRight+0.5)
            
    
    def setLeft( self ) :
        self._pwmDir   = self._pwmA
        self._pwmDir.duty( 0 )
        self._pwmSpeed = self._pwmB
    
    def setRight( self ) :
        self._pwmDir   = self._pwmB
        self._pwmDir.duty( 0 )
        self._pwmSpeed = self._pwmA
    
    def setSpeed( self, _fSpeed ) :
        if _fSpeed != self._fCurSpeed :
            iDuty = self.speedToDuty( _fSpeed )
                
            if _fSpeed > 0 and self._fCurSpeed <= 0 :
                self.setLeft()

            elif _fSpeed < 0 and self._fCurSpeed >= 0 :
                self.setRight()
        
            self._pwmSpeed.duty( iDuty )
        
        self._fCurSpeed = _fSpeed


### Sample

In [ ]:
from display import Display

class Gauge :
    def __init__(self, _display, _fValLeft, _fValRight, _iPosX, _iPosY, _iWidth, _iHeight, _iBorderWidth=1 ):
        self._display = _display
        self._fScale  = (_iWidth-2*_iBorderWidth)/(_fValRight-_fValLeft)
        self._iPosX   = _iPosX
        self._iPosY   = _iPosY
        self._iWidth  = _iWidth
        self._iHeight = _iHeight
        self._iBorderWidth = _iBorderWidth
        self._iPos0   = int(-_fValLeft*self._fScale + _iPosX+1.5)
        self._iVal    = 0
        self._fVal    = 0
        
    def setValue( self, _fVal ) :
        self._fVal = _fVal
        self._iVal = int(self._fScale*_fVal)
        
        self.paint()
        
    def paint() :
        pass
    
class BarGauge( Gauge ) :
    def __init__(self, _display, _fValLeft, _fValRight, _iPosX, _iPosY, _iWidth, _iHeight ):
        super().__init__(_display, _fValLeft, _fValRight, _iPosX, _iPosY, _iWidth, _iHeight )
        
    def paint( self ) :
        self._display.rect(self._iPosX,self._iPosY,self._iWidth,self._iHeight)
        
        if self._fVal >= 0 :
            self._display.fill_rect(self._iPos0,self._iPosY,self._iVal,self._iHeight)
        else:
            self._display.fill_rect(self._iVal+self._iPos0,self._iPosY,-self._iVal,self._iHeight)

        self._display.vline(self._iPos0,self._iPosY,self._iHeight)


class ThumbGauge( Gauge ) :
    def __init__(self, _display, _fValLeft, _fValRight, _iPosX, _iPosY, _iWidth, _iHeight ):
        super().__init__(_display, _fValLeft, _fValRight, _iPosX, _iPosY, _iWidth, _iHeight, 2 )

    def paint( self ) :
        self._display.rect(self._iPosX,self._iPosY,self._iWidth,self._iHeight)
        self._display.fill_rect(self._iPos0+self._iVal-self._iBorderWidth,self._iPosY,1+2*self._iBorderWidth,self._iHeight)
        self._display.vline(self._iPos0,self._iPosY,self._iHeight)
        

import time
from display import Display

display = Display()

DELAY_MILLIS = 0

fFrequency = 50
fSpeedMax  = 500
iSteps     = 10 # 128
fStep      = fSpeedMax/iSteps
iDutyMax   = 1023


gaugeSpeed = BarGauge  ( display, -fSpeedMax, fSpeedMax,  0, 12, 128, 10 )
        
motor = Motor( 12, 13, _iFrequency = fFrequency, _iDutyMaxLeft = iDutyMax, _iDutyMaxRight = iDutyMax, _fSpeedMaxLeft = fSpeedMax, _fSpeedMaxRight = fSpeedMax )

fSpeed = 0

for i in range( 10 ):
    while fSpeed <= fSpeedMax:
        motor.setSpeed( fSpeed )
        display.clear()
        gaugeSpeed.setValue( fSpeed )
        display.show()
        #time.sleep_ms(DELAY_MILLIS)
        fSpeed+=fStep

    while fSpeed >= -fSpeedMax:
        motor.setSpeed( fSpeed )
        display.clear()
        gaugeSpeed.setValue( fSpeed )
        display.show()
        #time.sleep_ms(DELAY_MILLIS)
        fSpeed-=fStep

        
motor.setSpeed( 0 )
display.clear()
gaugeSpeed.setValue( 0 )
display.show()   
    
print( "Done" )        